In [21]:
from pathlib import Path

import geopandas as gpd
import pandas as pd

In [22]:
YEAR = 2023

In [ ]:
data = pd.read_csv(
    f"../data/statistics/finland/009_12f7_{YEAR}_all.csv",
    skiprows=2,
    encoding="ISO-8859-1",
)

data["Information"] = data["Information"].str.replace(" (HR)", "")
data.rename({"Information": "information"}, axis=1, inplace=True)

data[["postal_code", "area_name"]] = data["Postal code area"].str.split(
    "  ", expand=True
)
data.drop(["Postal code area"], axis=1, inplace=True)

In [11]:
year = data[["postal_code", "area_name", "information", str(YEAR)]].copy()
year = pd.pivot_table(
    year, index=["postal_code"], columns=["information"], values=[str(YEAR)]
)
year.columns = year.columns.droplevel()
year.reset_index(inplace=True)
year = year.merge(
    data[["postal_code", "area_name"]].drop_duplicates(), on="postal_code"
)
year = year[["postal_code", "area_name"] + year.columns.tolist()[1:-1]].copy()

In [18]:
postal_areas = gpd.read_file(
    "../data/statistics/finland/finland_postalcode_area.geojson"
)
postal_areas = postal_areas[postal_areas["posti_alue"].isin(year["postal_code"])].copy()
postal_areas = postal_areas[
    ["posti_alue", "vuosi", "nimi", "pinta_ala", "geometry"]
].copy()
postal_areas.columns = ["postal_code", "year", "name", "surface_area", "geometry"]

In [19]:
merged = postal_areas.merge(year, on="postal_code")

In [23]:
Path(f"../data/statistics/helsinki/{YEAR}/").mkdir(parents=True, exist_ok=True)
merged.to_file(f"../data/statistics/helsinki/{YEAR}/statistics.geojson")

In [ ]:
# merged.plot(column="Average income of inhabitants")